# Set up for model experiments

We'll do the following here:

- Create distinct catalogs
- Document parameters changes that will accompany each
- Create yamls for each experiment

## Initial experiments - FTW baseline model, FTW dataset

The first tests will be on a few different parameters/settings on the existing FTW baseline model, on just the FTW dataset (the full one). 

### Catalog

In [2]:
import pandas as pd
from pathlib import Path
import yaml

catalog = pd.read_csv("../data/ftw-mappingafrica-combined-catalog.csv")
catalog.query("dataset == 'ftw'").to_csv("../data/ftw-catalog.csv", index=False)

### Config adjustor

In [31]:
# import yaml

# def write_yaml(template_path: str, output_path: str, updates: dict = None):
#     """
#     Write a YAML file from a template file, with optional updates.

#     Args:
#         template_path (str): Path to the base YAML template file.
#         output_path (str): Path to the output YAML file.
#         updates (dict, optional): Dictionary of keys/values to update.
#     """

#     def recursive_update(d, u):
#         for k, v in u.items():
#             if isinstance(v, dict) and isinstance(d.get(k), dict):
#                 recursive_update(d[k], v)
#             else:
#                 d[k] = v

#     with open(template_path, 'r') as f:
#         config = yaml.safe_load(f)
#         if updates:
#             recursive_update(config, updates)

#     class IndentDumper(yaml.SafeDumper):
#         def increase_indent(self, flow=False, indentless=False):
#             return super().increase_indent(flow, False)

#     with open(output_path, 'w') as f:
#         yaml.dump(
#             config,
#             f,
#             Dumper=IndentDumper,
#             default_flow_style=False,
#             sort_keys=False,
#             indent=2,
#             allow_unicode=True
#         )

def write_yaml(template_path: str, output_path: str, updates: dict = None):
    """
    Write a YAML file from a template file, with optional updates.

    Args:
        template_path (str): Path to the base YAML template file.
        output_path (str): Path to the output YAML file.
        updates (dict, optional): Dictionary of keys/values to update.
    """

    def recursive_update(d, u):
        for k, v in u.items():
            if isinstance(v, dict) and isinstance(d.get(k), dict):
                recursive_update(d[k], v)
            else:
                d[k] = v

    with open(template_path, 'r') as f:
        config = yaml.safe_load(f)
        if updates:
            recursive_update(config, updates)

    class IndentDumper(yaml.SafeDumper):
        def increase_indent(self, flow=False, indentless=False):
            return super().increase_indent(flow, False)

    # custom representer for lists
    def represent_list(dumper, data):
        # flow style only if all elements are scalars
        if all(isinstance(x, (str, int, float, bool, type(None))) for x in data):
            return dumper.represent_sequence("tag:yaml.org,2002:seq", data, 
                                             flow_style=True)
        else:
            return dumper.represent_sequence("tag:yaml.org,2002:seq", data, 
                                             flow_style=False)

    IndentDumper.add_representer(list, represent_list)

    with open(output_path, 'w') as f:
        yaml.dump(
            config,
            f,
            Dumper=IndentDumper,
            default_flow_style=False,  # keep dicts block-style
            sort_keys=False,
            indent=2,
            allow_unicode=True
        )


In [ ]:
template_path = "../configs/ftwbaseline-exp1.yaml"
with open(template_path, 'r') as f:
    config = yaml.safe_load(f)

config

### Experiments

All experiments here are FTW baseline model, window B only, on the FTW dataset.

Single parameter or no change:

1. FTW defaults (for comparison with FTW's results)
2. Locally-weighted tversky focal loss
3. min-max normalization, lab
4. min-max normalization, gab
5. photometric augmentation package
6. satslidemix
7. rescale

#### Setup

Below we set up a yaml for each experiment. Provide the following:

- `cfg_name`: name of the config/experiment file (without .yaml)
- `update`: dictionary of changes to make to the base config

Also define a global `home_dir` for the path to the repo containing the catalog. That's done once in the first cell. 

#### # 1

In [45]:
home_dir = "~/projects"
cfg_name = "ftwbaseline-exp1"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog.csv",
        )
    )
)
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### # 2


In [47]:
cfg_name = "ftwbaseline-localtversky-exp2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### # 3


In [48]:
cfg_name = "ftwbaseline-minmax_lab-exp3"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    global_stats=None,
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### # 4

Not yet run (need to calculate global stats)


#### # 5

In [49]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-photometric-exp5"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["brightness", "contrast", "gaussian_noise"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### # 6

In [50]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-satslide-exp6"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["satslidemix"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### # 7

In [51]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-rescale-exp7"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["rescale"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)